In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 1000 # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-1000.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:59:36<00:00,  7.18s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.249891e+14               0.0      7.164278
std    2.165959e+14               0.0      1.811283
min    6.677052e+03               0.0      5.159213
25%    8.147731e+03               0.0      5.879934
50%    9.319791e+03               0.0      6.702751
75%    1.249802e+14               0.0      7.822459
max    4.999892e+14               0.0     20.008771
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 80    # weight of constraint violation penealty
hlayers_sol = 4        # number of hidden layers for solution mapping
hlayers_rnd = 4        # number of hidden layers for solution mapping
hsize = 2048           # width of hidden layers for solution mapping
lr = 1e-3              # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 209522.95
Epoch 1, Validation Loss: 180980.51
Epoch 2, Validation Loss: 76112.93
Epoch 3, Validation Loss: 443439.25
Epoch 4, Validation Loss: 39903.96
Epoch 5, Validation Loss: 45483.47
Epoch 6, Validation Loss: 29610.77
Epoch 7, Validation Loss: 47120.95
Epoch 8, Validation Loss: 22747.78
Epoch 9, Validation Loss: 1233025546.35
Epoch 10, Validation Loss: 31109.25
Epoch 11, Validation Loss: 23331.81
Epoch 12, Validation Loss: 27422.60
Epoch 13, Validation Loss: 16699.41
Epoch 14, Validation Loss: 24157.16
Epoch 15, Validation Loss: 18064.45
Epoch 16, Validation Loss: 22602.64
Epoch 17, Validation Loss: 21087.52
Epoch 18, Validation Loss: 12535.24
Epoch 19, Validation Loss: 12371.85
Epoch 20, Validation Loss: 13072.17
Epoch 21, Validation Loss: 1394057239.77
Epoch 22, Validation Loss: 32363.06
Epoch 23, Validation Loss: 25055.87
Epoch 24, Validation Loss: 12680.14
Epoch 25, Validation Loss: 11976.16
Epoch 26, Validation Loss: 2368721255.04
Epoch 27, Validation

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-1000.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:34<00:00,  6.47it/s]


            Obj Val  Constraints Viol  Elapsed Time
count   1000.000000       1000.000000   1000.000000
mean    6650.374215          1.307318      0.019679
std     1120.364047         11.214535      0.002546
min     3792.878020          0.000000      0.003506
25%     6100.835013          0.000000      0.019606
50%     6499.965338          0.000000      0.020024
75%     7239.633277          0.000000      0.020530
max    11121.786878        177.610756      0.023118
Number of infeasible solution: 21


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 2048          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 185825.69
Epoch 1, Validation Loss: 105174.41
Epoch 2, Validation Loss: 98083.05
Epoch 3, Validation Loss: 134239.02
Epoch 4, Validation Loss: 48417.94
Epoch 5, Validation Loss: 29058.18
Epoch 6, Validation Loss: 26681.46
Epoch 7, Validation Loss: 227183.36
Epoch 8, Validation Loss: 7782363525.31
Epoch 9, Validation Loss: 24340.26
Epoch 10, Validation Loss: 67583.60
Epoch 11, Validation Loss: 21142.05
Epoch 12, Validation Loss: 19603.90
Epoch 13, Validation Loss: 23721849743.30
Epoch 14, Validation Loss: 24693.95
Epoch 15, Validation Loss: 143229324.37
Epoch 16, Validation Loss: 2910652813205272.00
Epoch 17, Validation Loss: 855063369200539604615168.00
Epoch 18, Validation Loss: 14565.40
Epoch 19, Validation Loss: 27198.74
Epoch 20, Validation Loss: 15129.94
Epoch 21, Validation Loss: 23339.76
Epoch 22, Validation Loss: 18995.59
Epoch 23, Validation Loss: 21900.59
Epoch 24, Validation Loss: 11159.74
Epoch 25, Validation Loss: 12761.09
Epoch 26, Validation Loss

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-1000.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:24<00:00,  6.90it/s]


            Obj Val  Constraints Viol  Elapsed Time
count   1000.000000       1000.000000   1000.000000
mean    6195.200172          1.821974      0.019639
std     1562.808182         21.032867      0.002739
min     3965.215929          0.000000      0.003000
25%     4878.100643          0.000000      0.019650
50%     5876.587116          0.000000      0.020029
75%     7376.366106          0.000000      0.020485
max    11598.931829        450.352139      0.034322
Number of infeasible solution: 19
